In [1]:
import polars as pl
schema_overrides = {f'Col{i:02d}': pl.Float64 for i in range(1, 51)}
df_datos = pl.read_csv("SerieA2023.csv", separator=';', schema_overrides=schema_overrides)
df_region = df_datos.filter(pl.col('IdRegion') == 7)


In [2]:
df_region.write_parquet('SerieA2023Maule.parquet')

In [3]:
import pandas as pd
import numpy as np
# Leer el archivo Excel
df_diccionario = pd.read_excel("A01.xlsx", skiprows=5)

# Rellenar valores faltantes hacia adelante
df_diccionario = df_diccionario.ffill(axis=0)
df_diccionario = df_diccionario.ffill(axis=1)

# Eliminar filas con menos de 3 valores no nulos
df_diccionario.dropna(axis=0, thresh=3, inplace=True)

# Limitar el DataFrame a las primeras 25 filas
df_corte = df_diccionario.iloc[:25]

# Establecer la primera fila como los nombres de las columnas y eliminar esa fila del DataFrame
df_corte.columns = df_corte.iloc[0]
df_corte = df_corte[1:]

# Renombrar las primeras columnas
listacol = df_corte.columns.tolist()
listacol[0] = 'Codigo Prestacion'
listacol[1] = 'Tipo Control'
listacol[2] = 'Profesional'
df_corte.columns = listacol

df_corte = df_corte.iloc[:, :30]
df_corte.iloc[0, 1] = np.nan
df_corte.iloc[0, 2] = np.nan
df_corte.iloc[1, 1] = np.nan
df_corte.iloc[1, 2] = np.nan

df_caract = df_corte.iloc[:3]
df_corte = df_corte.iloc[2:]



In [4]:
df_caract = df_caract
df_caract.iloc[0, 1] = np.nan
df_caract.iloc[0, 2] = np.nan
df_caract.iloc[1, 1] = np.nan
df_caract.iloc[1, 2] = np.nan
df_caract.iloc[2, 0] = np.nan
df_caract.iloc[2, 1] = np.nan
df_caract.iloc[2, 2] = np.nan
df_caract.reset_index(drop=True, inplace=True)

df_caract = df_caract.T
# Renombrar las primeras columnas
listacol = df_caract.columns.tolist()
listacol[0] = 'Tipo de Caracteristica'
listacol[1] = 'Caracteristica'
listacol[2] = 'Atributo'
df_caract.columns = listacol

df_caract = df_caract.iloc[3:]   

In [5]:
df_melted = pd.melt(df_corte, id_vars=['Codigo Prestacion', 'Tipo Control', 'Profesional'], var_name='Columna', value_name='Atributo')

# Crear mapeos para tipos de características y características basados en los atributos
tipo_caracteristica_map = df_caract.set_index('Atributo')['Tipo de Caracteristica'].to_dict()
caracteristica_map = df_caract.set_index('Atributo')['Caracteristica'].to_dict()

# Mapear los tipos de características y características desde df_caract
df_melted['Tipo de Caracteristica'] = df_melted['Atributo'].map(tipo_caracteristica_map)
df_melted['Caracteristica'] = df_melted['Atributo'].map(caracteristica_map)
# Agregar la columna 'Seccion'
df_melted['Seccion'] = 'SECCIÓN A: CONTROLES DE SALUD SEXUAL Y REPRODUCTIVA'

# Reordenar las columnas
df_melted = df_melted[['Seccion', 'Codigo Prestacion', 'Tipo Control', 'Profesional', 'Tipo de Caracteristica', 'Caracteristica', 'Atributo']]
df_melted.dropna(subset=['Codigo Prestacion'], inplace=True)

# Crear la nueva columna 'REM' con valor 'REM01'
df_melted['REM'] = 'REM01'

# Reordenar las columnas para que 'REM' quede como la primera columna
df_melted = df_melted[['REM', 'Seccion', 'Codigo Prestacion', 'Tipo Control', 'Profesional', 'Tipo de Caracteristica', 'Caracteristica', 'Atributo']]

#df melted a polar
df_melted = pl.DataFrame(df_melted)
df_melted 

REM,Seccion,Codigo Prestacion,Tipo Control,Profesional,Tipo de Caracteristica,Caracteristica,Atributo
str,str,str,str,str,str,str,str
"""REM01""","""SECCIÓN A: CONTROLES DE SALUD …","""01010103""","""Pre-Concepcional""","""Matrona/ón""","""TOTAL ""","""TOTAL ""","""COL01"""
"""REM01""","""SECCIÓN A: CONTROLES DE SALUD …","""01010201""","""Prenatal""","""Médico/a""","""TOTAL ""","""TOTAL ""","""COL01"""
"""REM01""","""SECCIÓN A: CONTROLES DE SALUD …","""01010203""","""Prenatal""","""Matrona/ón""","""TOTAL ""","""TOTAL ""","""COL01"""
"""REM01""","""SECCIÓN A: CONTROLES DE SALUD …","""01501050""","""Post Parto""","""Médico/a""","""TOTAL ""","""TOTAL ""","""COL01"""
"""REM01""","""SECCIÓN A: CONTROLES DE SALUD …","""01501060""","""Post Parto""","""Matrona/ón""","""TOTAL ""","""TOTAL ""","""COL01"""
…,…,…,…,…,…,…,…
"""REM01""","""SECCIÓN A: CONTROLES DE SALUD …","""01010603""","""Ginecológico""","""Matrona/ón""","""Migrantes""","""Migrantes""","""COL27"""
"""REM01""","""SECCIÓN A: CONTROLES DE SALUD …","""01010901""","""Climaterio""","""Médico/a""","""Migrantes""","""Migrantes""","""COL27"""
"""REM01""","""SECCIÓN A: CONTROLES DE SALUD …","""01010903""","""Climaterio""","""Matrona/ón""","""Migrantes""","""Migrantes""","""COL27"""


In [8]:
df_region

Mes,IdServicio,Ano,IdEstablecimiento,CodigoPrestacion,IdRegion,IdComuna,Col01,Col02,Col03,Col04,Col05,Col06,Col07,Col08,Col09,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,Col41,Col42,Col43,Col44,Col45,Col46,Col47,Col48,Col49,Col50
i64,i64,i64,i64,str,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
8,16,2023,116111,"""04025010""",7,7201,5.0,2.0,3.0,null,null,null,null,null,null,null,null,null,1.0,2.0,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null
7,16,2023,116829,"""08230802""",7,7201,1034.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
7,16,2023,116829,"""08230803""",7,7201,13.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
7,16,2023,116829,"""08230812""",7,7201,1.0,0.0,1.0,null,null,null,null,null,null,null,null,null,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
7,16,2023,116829,"""08224000""",7,7201,14.0,12.0,2.0,null,null,3.0,null,null,null,2.0,null,5.0,null,1.0,1.0,1.0,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9,16,2023,116306,"""27280400""",7,7101,33.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,9.0,11.0,12.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
9,16,2023,116306,"""27500250""",7,7101,16.0,null,null,null,null,null,null,null,null,null,null,null,null,5.0,5.0,4.0,2.0,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
9,16,2023,116306,"""27300918""",7,7101,30.0,null,null,null,null,null,30.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [9]:
import polars as pl

# Identificar columnas dinámicas que comienzan con 'Col'
columnas_dinamicas = [col for col in df_region.columns if col.startswith('Col')]

# Transformar df_datos a formato largo, conservando las columnas relevantes
df_region = df_region.melt(
    id_vars=['Mes', 'IdServicio', 'Ano', 'IdEstablecimiento', 'CodigoPrestacion', 'IdRegion', 'IdComuna'],
    value_vars=columnas_dinamicas,
    variable_name='Variable',
    value_name='Cantidad'
)

# Obtener valores únicos de la columna 'IdEstablecimiento'
id_establecimientos_unicos = df_region.select('IdEstablecimiento').unique()

# Mostrar los valores únicos

#print(id_establecimientos_unicos)

# Saber si el establecimiento 116101 existe en el DataFrame
existe_establecimiento = 116101 in id_establecimientos_unicos['IdEstablecimiento']
#print(existe_establecimiento)

# Filtrar el DataFrame para el IdEstablecimiento específico
df_region_filtered = df_region.filter(pl.col('IdEstablecimiento') == 116101)

# Agregar columna 'PK' al DataFrame filtrado
df_region_filtered = df_region_filtered.with_columns(
    (
        pl.col('IdEstablecimiento').cast(str) +
        pl.col('CodigoPrestacion').cast(str).str.strip_chars('0') +  # Usar strip_chars para eliminar ceros a la izquierda
        pl.col('Variable').str.to_uppercase()
    ).alias('PK')
)
for IdEstablecimiento in df_region_filtered['IdEstablecimiento'].unique().to_list():
    df_melted = df_melted.with_columns(
        (
            pl.lit(IdEstablecimiento).cast(str) +
            pl.col('Codigo Prestacion').cast(str).str.strip_chars('0') +  # Usar strip_chars para eliminar ceros a la izquierda
            pl.col('Atributo').str.to_uppercase()
        ).alias('PK')
    )

# Crear una lista para almacenar resultados parciales
result_list = []

# Iterar sobre cada 'IdEstablecimiento'
for id_establecimiento in id_establecimientos_unicos['IdEstablecimiento'].to_list():
   #print(f'Procesando establecimiento {id_establecimiento}...')

    # Filtrar el DataFrame para el establecimiento actual
    df_melted = df_region_filtered.filter(pl.col('IdEstablecimiento') == id_establecimiento)

    # Crear la columna 'PK' en df_melted
    df_melted = df_melted.with_columns(
        (
            pl.lit(id_establecimiento).cast(str) +
            pl.col('CodigoPrestacion').cast(str).str.strip_chars('0') +  # Usar strip_chars para eliminar ceros a la izquierda
            pl.col('Variable').str.to_uppercase()
        ).alias('PK')
    )

    # Realizar la combinación directamente
    df_partial = df_region_filtered.join(df_melted, on='PK', how='inner')

    # Agregar el resultado parcial a la lista
    result_list.append(df_partial)

# Concatenar todos los resultados parciales en un solo DataFrame
df_final = pl.concat(result_list, how='vertical')


C:\Users\jsotoletelier\AppData\Local\Temp\ipykernel_12088\861509459.py:7: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df_region = df_region.melt(


In [10]:
df_final

Mes,IdServicio,Ano,IdEstablecimiento,CodigoPrestacion,IdRegion,IdComuna,Variable,Cantidad,PK,Mes_right,IdServicio_right,Ano_right,IdEstablecimiento_right,CodigoPrestacion_right,IdRegion_right,IdComuna_right,Variable_right,Cantidad_right
i64,i64,i64,i64,str,i64,i64,str,f64,str,i64,i64,i64,i64,str,i64,i64,str,f64
4,16,2023,116101,"""23090620""",7,7308,"""Col01""",10.0,"""1161012309062COL01""",4,16,2023,116101,"""23090620""",7,7308,"""Col01""",10.0
10,16,2023,116101,"""23090620""",7,7308,"""Col01""",96.0,"""1161012309062COL01""",4,16,2023,116101,"""23090620""",7,7308,"""Col01""",10.0
9,16,2023,116101,"""23090620""",7,7308,"""Col01""",76.0,"""1161012309062COL01""",4,16,2023,116101,"""23090620""",7,7308,"""Col01""",10.0
11,16,2023,116101,"""23090620""",7,7308,"""Col01""",114.0,"""1161012309062COL01""",4,16,2023,116101,"""23090620""",7,7308,"""Col01""",10.0
1,16,2023,116101,"""23090620""",7,7308,"""Col01""",14.0,"""1161012309062COL01""",4,16,2023,116101,"""23090620""",7,7308,"""Col01""",10.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
5,16,2023,116101,"""01110107""",7,7308,"""Col50""",null,"""1161011110107COL50""",6,16,2023,116101,"""01110107""",7,7308,"""Col50""",null
9,16,2023,116101,"""01110107""",7,7308,"""Col50""",null,"""1161011110107COL50""",6,16,2023,116101,"""01110107""",7,7308,"""Col50""",null
7,16,2023,116101,"""01110107""",7,7308,"""Col50""",null,"""1161011110107COL50""",6,16,2023,116101,"""01110107""",7,7308,"""Col50""",null


In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Crear una conexión a la base de datos PostgreSQL
engine = create_engine('postgresql+psycopg2://postgres:@localhost:5432/REM')

# Supongamos que df_final es tu DataFrame de Polars convertido a Pandas
df_final = df_final.to_pandas()

# Enviar los datos a una tabla en PostgreSQL
df_final.to_sql('A01SA_REM', engine, if_exists='replace', index=False)

print("Datos enviados a la base de datos PostgreSQL con éxito.")

In [ ]:
#IMPORTANTE NO CORRER ESTA CELDA SI NO ES PARA TODOS LOS ESTABLECIMIENTOS
#funcion para todos los establecimientos

import polars as pl

# Agregar columna 'PK' al DataFrame original de una vez
df_region = df_region.with_columns(
    (
        pl.col('IdEstablecimiento').cast(str) +
        pl.col('CodigoPrestacion').cast(str).str.strip_chars('0') +  # Usar strip_chars para eliminar ceros a la izquierda
        pl.col('Variable').str.to_uppercase()
    ).alias('PK')
)

# Obtener los valores únicos de 'IdEstablecimiento'
id_establecimientos_unicos = df_region.select('IdEstablecimiento').unique()

# Crear una lista para almacenar resultados parciales
result_list = []

# Iterar sobre cada 'IdEstablecimiento'
for id_establecimiento in id_establecimientos_unicos['IdEstablecimiento'].to_list():
    print(f'Procesando establecimiento {id_establecimiento}...')

    # Filtrar el DataFrame para el establecimiento actual
    df_melted = df_region.filter(pl.col('IdEstablecimiento') == id_establecimiento)

    # Realizar la combinación directamente
    df_partial = df_region.join(df_melted, on='PK', how='inner')

    # Agregar el resultado parcial a la lista
    result_list.append(df_partial)

# Concatenar todos los resultados parciales en un solo DataFrame
df_final = pl.concat(result_list, how='vertical')

# Mostrar el resultado final
print(df_final)


In [ ]:
#como seria el sql de mi tabla en postgresql
'''
CREATE TABLE "A01SA_REM" (
    "Mes" INT,
    "IdServicio" INT,
    "Ano" INT,
    "IdEstablecimiento_x" VARCHAR(50),
    "CodigoPrestacion" VARCHAR(50),
    "IdRegion" INT,
    "IdComuna" INT,
    "Variable" VARCHAR(50),
    "Cantidad" INT,
    "PK" VARCHAR(50),
    "Seccion" VARCHAR(255),
    "CODIGO_PRESTACION" VARCHAR(50),
    "TIPO_CONTROL" VARCHAR(50),
    "PROFESIONAL" VARCHAR(50),
    "Tipo_de_Caracteristica" VARCHAR(255),
    "Caracteristica" VARCHAR(255),
    "Atributo" VARCHAR(255),
);

select * from "A01SA_REM";
'''